In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed bhi
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
ad

In [3]:
!pip install joblib

In [4]:
!pip install scikit-learn-intelex

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 MB 6.1 MB/s eta 0:00:00


In [5]:
# plugin to sklearn for supporting gpu and leveraging gpu while training
from sklearnex import patch_sklearn
patch_sklearn()


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [6]:
# from google.colab import userdata
# userdata.get('secretName')
project_dir = '/content/drive/MyDrive/mrunals_project'
# project_dir = r'D:\dev\shriekdj\mrunal_related_project_for_workload_data'

In [7]:
zip_path = os.path.join(project_dir, 'converted_to_csv_again.zip')
zip_path

'/content/drive/MyDrive/mrunals_project/converted_to_csv_again.zip'

In [8]:
import pandas as pd
import zipfile
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

## Wait Time Optimization Model ( Using Gradient Boosting )

This model predicts the wait time for a job to start based on its resource requirements and other features.

In [9]:
from typing import Literal

# Function to load all CSVs from a zip into a single DataFrame without needing to extract non nesessory columns
def get_df(file, model_type: str):
  df = pd.read_csv(file)
  columns = list(df.columns)
  columns_only = list()
  columns_to_check = list()
  match model_type:
    case 'resource':
      columns_to_check = [
          'Submit Time',
          'Average CPU Time Used',
          'Requested Number of Processors',
          'Requested Time',
          # 'User ID',
          # 'Group ID',
          'Allocated Processors'
      ]
    case 'wait':
      columns_to_check = [
          'Allocated Processors',
          'Requested Number of Processors',
          'Submit Time',
          # 'User ID',
          'Wait Time'
      ]
    case 'energy':
      columns_to_check = [
          'Used Memory',
          'Run Time',
          'Allocated Processors'
      ]
    case _:
      raise Exception('Invalid Model Type')

  for col in columns_to_check:
    if col in columns:
      columns_only.append(col)
  return df[columns_only]

def load_data_from_zip(zip_path: str, model_type: str):
    with zipfile.ZipFile(zip_path, 'r') as z:
        # List all file names in the zip that end with .csv
        csv_files = [f for f in z.namelist() if f.endswith('.csv')]
        # Load each CSV file directly from the zip and concatenate into one DataFrame
        df_list = [get_df(z.open(file), model_type) for file in csv_files]
        full_df = pd.concat(df_list, ignore_index=True)
        full_df.dropna(
            # subset=['Submit Time', 'Average CPU Time Used', 'Requested Number of Processors', 'Requested Time', 'User ID', 'Group ID', 'Allocated Processors'],
            inplace=True
          )
        return full_df

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import os, joblib

### This Section is for Training and Testing From Start

In [11]:
from sklearn.ensemble import RandomForestRegressor

df_wait_cleaned = load_data_from_zip(zip_path, 'wait')

In [14]:
df_wait_cleaned
# df_wait_cleaned.to_csv('wait_cleaned.csv', index=False)

,Allocated Processors,Requested Number of Processors,Submit Time,Wait Time
300000,0.000000,0.000000,0.000000,9.261504e-07
300001,0.000000,0.000000,0.000003,9.854241e-05
300002,0.055118,0.055118,0.000019,2.918670e-03
300003,0.244094,0.244094,0.000080,3.995043e-03
300004,0.047244,0.047244,0.000130,8.607642e-04
...,...,...,...,...
68340819,0.277670,0.277670,0.999737,2.008016e-06
68340820,0.114563,0.114563,0.999817,1.004008e-06
68340821,0.000000,0.000000,0.999995,4.016032e-06
68340822,0.000000,0.000000,0.999995,1.004008e-06


In [ ]:
# Proceed with your existing feature selection and model training steps
X_wait = df_wait_cleaned[
    [
        'Allocated Processors',
        'Requested Number of Processors',
        'Submit Time'
        # 'User ID',
    ]
]
y_wait = df_wait_cleaned['Wait Time'].astype(float)

In [ ]:
del df_wait_cleaned

In [ ]:
# Split the data
X_train_wait, X_test_wait, y_train_wait, y_test_wait = train_test_split(
    X_wait, y_wait, test_size=0.2, random_state=42
)

In [ ]:
# removing the variable from memory which are not in use from now
del X_wait
del y_wait

In [ ]:
y_test_wait

,Wait Time
15284823,4.862529e-06
12393122,1.031702e-05
2531255,1.871910e-06
5007918,4.239326e-06
13306598,3.382629e-07
...,...
10983930,3.273539e-04
17014173,6.274776e-05
11855624,1.057071e-06
16164944,4.651114e-07


In [ ]:
# Backing up test and training data ( It Not Saved on Google Drive But this Colab Files Navigation Icon)
joblib.dump(X_train_wait, 'X_train_wait.joblib')
joblib.dump(X_test_wait, 'X_test_wait.joblib')
joblib.dump(y_train_wait, 'y_train_wait.joblib')
joblib.dump(y_test_wait, 'y_test_wait.joblib')

['y_test_wait.joblib']

In [ ]:
# saving Them on Google Drive Within Zip File
with zipfile.ZipFile(
      os.path.join(
          project_dir,
          'model_wait_training_and_test_files.zip'
      ),
      'w',
      zipfile.ZIP_DEFLATED
    ) as zip:
    # Writing each file one by one
    for file in [
        'X_train_wait.joblib',
        'X_test_wait.joblib',
        'y_train_wait.joblib',
        'y_test_wait.joblib'
    ]:
        print(file)
        zip.write(os.path.join(os.getcwd(), file), arcname=file)

X_train_wait.joblib
X_test_wait.joblib
y_train_wait.joblib
y_test_wait.joblib


### This Section is for Loading Training and Testing Backup

In [ ]:
import joblib
# Restoring Backed Up Models
with zipfile.ZipFile(
      os.path.join(
        project_dir, 'model_wait_training_and_test_files.zip'
      ),
      'r'
    ) as z:
    # Load each CSV file directly from the zip and concatenate into one DataFrame
    X_train_wait = joblib.load(
        z.open('X_train_wait.joblib')
    )
    X_test_wait = joblib.load(
        z.open('X_test_wait.joblib')
    )
    y_train_wait = joblib.load(
        z.open('y_train_wait.joblib')
    )
    y_test_wait = joblib.load(
        z.open('y_test_wait.joblib')
    )
    # full_df = pd.concat(df_list, ignore_index=True)

### Section for Training The Models

In [ ]:
# Initialize and train the model with cpu
# model_wait = RandomForestRegressor(n_estimators=100, random_state=42)
model_wait = GradientBoostingRegressor(random_state=42)
model_wait.fit(X_train_wait, y_train_wait)

GradientBoostingRegressor(random_state=42)

In [ ]:
del X_train_wait
del y_train_wait

In [ ]:
# Predict and evaluate the model
predictions_wait = model_wait.predict(X_test_wait)   #Predict function accepts array values or pandas.dataframe , prediction_wait is similar to python list type NDARRAY
mse_wait = mean_squared_error(y_test_wait, predictions_wait)
print('RMSE for wait Requirement Model:', np.sqrt(mse_wait))

RMSE for wait Requirement Model: 0.011287189751268424


In [ ]:
# saving model to google Drive as joblib file to use in future
import joblib
joblib.dump(
    model_wait,
    os.path.join(project_dir, 'model_wait.joblib')
)

['/content/drive/MyDrive/mrunals_project/model_wait.joblib']